# Notebook de calcul d'indicateurs sur les données GTFS

In [ ]:
# A exécuter dans Colab !
!git clone https://github.com/CEREMA/hackathon-gtfs
%cd hackathon-gtfs
!pip install gtfs_kit

Cette case importe :
* la librairie gtfs_kit, utilisé en partie pour le calcul des indicateurs,
* des librairies essentielles pour le traitement des données,
* des fonctions développées spécifiquement pour les besoins de calcul des indicateurs.

In [1]:
import sys

sys.path.append('..')

import gtfs_kit as gk
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString

import folium
from folium import plugins
import numpy as np
import branca.colormap as cm

from src.utils import (
    charger_gtfs,
    obtenir_service_ids_pour_date,
    exporter_df_to_csv,
    exporter_geojson,
    exporter_gdf_to_csv
)
from src.arrets import calculer_indicateurs_arrets, afficher_statistiques
from src.cartographie import creer_carte_troncons, create_carte_arrets
from src.create_troncons_uniques import creer_troncons_uniques
from src.indicateurs_troncons import compute_indicateurs_troncons

## Configuration des calculs
Ces cases permettent de définir le chemin vers le zip GTFS à analyser, ainsi que la date pour laquelle les indicateurs doivent être calculés.

In [2]:
# Chemin vers le zip GTFS
GTFS_ZIP_PATH = "data/TAM_MMM_GTFS.zip"

# La librairie gtfs_kit est utilisée pour charger le GTFS
feed = charger_gtfs(GTFS_ZIP_PATH)

Chargement du fichier GTFS : data/TAM_MMM_GTFS.zip
✓ GTFS chargé avec succès


In [3]:
# Saisir la date pour laquelle on veut récupérer les données :
DATE_ANALYSE = "20251123"  # Format AAAAMMJJ

# Les services actifs à la date choisie sont calculés
active_service_ids = obtenir_service_ids_pour_date(feed, DATE_ANALYSE)

✓ Services actifs le 20251123 : 2 service(s)


## Arrêts

### Calcul des indicateurs

La cellule suivante est une copie de la routine de création des indicateurs aux arrêts. Il n'est pas nécessaire de l'exécuter, mais vous pouvez la consulter et la modifier si besoin.

In [ ]:
def calculer_indicateurs_arrets(feed, active_service_ids: list[str], date_str: str):
    """
    Calcule les indicateurs pour chaque arrêt :
    - Nombre de lignes desservies
    - Nombre de passages
    - Heure du premier départ
    - Heure du dernier départ
    - Amplitude horaire
    - Temps d'attente moyen, min et max
    """
    print(f"\nCalcul des indicateurs aux arrêts...")

    if not active_service_ids:
        print("⚠ Aucun service actif pour cette date")
        return None

    # Filtrer les trips actifs ce jour-là
    trips_actifs = feed.trips[feed.trips["service_id"].isin(active_service_ids)]
    print(f"✓ {len(trips_actifs)} trips actifs")

    # Joindre avec stop_times
    stop_times_actifs = feed.stop_times.merge(
        trips_actifs[["trip_id", "service_id"]], on="trip_id"
    )

    # Calculer les indicateurs par arrêt
    indicateurs = (
        stop_times_actifs.groupby("stop_id")
        .agg(
            nombre_passages=("trip_id", "count"),
            premier_depart=("departure_time", "min"),
            dernier_depart=("departure_time", "max"),
        )
        .reset_index()
    )

    # Utilisation de gtfs_kit pour calculer les stats de headway
    indic_gk = feed.compute_stop_stats([date_str])
    indicateurs = indicateurs.merge(
        indic_gk[["stop_id", "mean_headway", "max_headway", "num_routes"]],
        on="stop_id",
        how="left",
    )
    indicateurs.rename(
        columns={
            "num_routes": "nb_lignes",
            "mean_headway": "temps_attente_moyen",
            "max_headway": "temps_attente_max",
        },
        inplace=True,
    )

    # Joindre avec les informations des arrêts
    indicateurs = indicateurs.merge(
        feed.stops[["stop_id", "stop_name", "stop_lat", "stop_lon"]],
        on="stop_id",
        how="left",
    )

    # Calculer l'amplitude horaire
    indicateurs["amplitude_horaire"] = pd.to_timedelta(
        indicateurs["dernier_depart"]
    ) - pd.to_timedelta(indicateurs["premier_depart"])
    # nettoyer pour retirer le 0 days
    indicateurs["amplitude_horaire"] = indicateurs["amplitude_horaire"].apply(
        lambda x: str(x).replace("0 days ", " ").strip()
    )

    # Réorganiser les colonnes
    indicateurs = indicateurs[
        [
            "stop_id",
            "stop_name",
            "stop_lat",
            "stop_lon",
            "nb_lignes",
            "nombre_passages",
            "premier_depart",
            "dernier_depart",
            "amplitude_horaire",
            "temps_attente_moyen",
            "temps_attente_max",
        ]
    ]

    # Trier par nombre de passages décroissant
    indicateurs = indicateurs.sort_values("nombre_passages", ascending=False)

    print(f"✓ Indicateurs calculés pour {len(indicateurs)} arrêts")

    return indicateurs

In [4]:
indicateurs = calculer_indicateurs_arrets(feed, active_service_ids, DATE_ANALYSE)

# Il est possible d'exporter les résultats en csv
exporter_df_to_csv(
    indicateurs,
    f"output/indicateurs_arrets_{DATE_ANALYSE}.csv"
)


Calcul des indicateurs aux arrêts...
✓ 1568 trips actifs
✓ Indicateurs calculés pour 1388 arrêts
✓ CSV exporté : output/indicateurs_arrets_20251123.csv


La cellule suivante est une copie de la routine d'affichage des statistiques aux arrêts. Il n'est pas nécessaire de l'exécuter, mais vous pouvez la consulter et la modifier si besoin.

In [ ]:
def afficher_statistiques(df):
    """Affiche des statistiques résumées"""
    print("\n" + "=" * 70)
    print("STATISTIQUES GLOBALES")
    print("=" * 70)
    print(f"Nombre total d'arrêts desservis : {len(df)}")
    print(f"Nombre total de passages : {df['nombre_passages'].sum():,}")
    print(f"Moyenne de passages par arrêt : {df['nombre_passages'].mean():.1f}")
    print(f"Médiane de passages par arrêt : {df['nombre_passages'].median():.1f}")
    print(
        f"\nArrêt le plus fréquenté : {df.iloc[0]['stop_name']} ({df.iloc[0]['nombre_passages']} passages)"
    )
    print(f"Premier départ global : {df['premier_depart'].min()}")
    print(f"Dernier départ global : {df['dernier_depart'].max()}")

    print("\n" + "=" * 70)
    print("TOP 10 DES ARRÊTS LES PLUS FRÉQUENTÉS")
    print("=" * 70)
    print(df.head(10).to_string(index=False))

In [5]:
afficher_statistiques(indicateurs)


STATISTIQUES GLOBALES
Nombre total d'arrêts desservis : 1388
Nombre total de passages : 33,351
Moyenne de passages par arrêt : 24.0
Médiane de passages par arrêt : 15.0

Arrêt le plus fréquenté : Mosson (175 passages)
Premier départ global : 05:36:00
Dernier départ global : 25:51:00

TOP 10 DES ARRÊTS LES PLUS FRÉQUENTÉS
stop_id                 stop_name  stop_lat  stop_lon  nb_lignes  nombre_passages premier_depart dernier_depart amplitude_horaire  temps_attente_moyen  temps_attente_max
   1130                    Mosson 43.616320  3.819787        1.0              175       05:42:00       25:47:00          20:05:00             6.459459               15.0
   1153 Moularès (Hôtel de Ville) 43.600730  3.895359        2.0              152       06:19:00       24:38:00          18:19:00             6.922330               15.0
   1163             Port Marianne 43.601565  3.899180        2.0              152       07:00:00       25:20:00          18:20:00             6.990291               1

### Représentation cartographique

In [ ]:
def create_carte_arrets(df):
    # Carte des arrêts avec leur nombre de passages

    # Définir les seuils pour les couleurs
    passages_values = df["nombre_passages"].values
    bins = np.percentile(passages_values, [0, 25, 50, 75, 100])
    bins = [0] + list(bins[1:])  # Assurer que le minimum est 0

    def get_color(passages):
        if passages == 0:
            return "gray"
        elif passages <= bins[1]:
            return "green"
        elif passages <= bins[2]:
            return "yellow"
        elif passages <= bins[3]:
            return "orange"
        else:
            return "red"

    m = folium.Map(
        location=[df["stop_lat"].mean(), df["stop_lon"].mean()],
        zoom_start=12,
        width="100%",
        height="500px",
    )

    for _, row in df.iterrows():
        stop_id = row["stop_id"]
        lat = row["stop_lat"]
        lon = row["stop_lon"]
        passages = row["nombre_passages"]

        color = get_color(passages)

        folium.CircleMarker(
            location=[lat, lon],
            radius=2,
            popup=f"Arrêt ID: {stop_id}\nPassages: {passages}",
            color=color,
            fill=True,
            fill_color=color,
        ).add_to(m)

    m.save("stops_map.html")

    print("\n✓ Carte des arrêts enregistrée dans : stops_map.html")

    return m

In [6]:
carte_arrets = create_carte_arrets(indicateurs)
carte_arrets

In [ ]:
# (Optionnel) Sauvegarder en HTML
carte_arrets.save(f'output/ma_carte_arrets_{DATE_ANALYSE}.html')

## Tronçons

Quand le fichier shapes.txt n'est pas disponible dans le zip GTFS, les calculs automatiques sur les tronçons sont impossibles. Il est nécessaire de les créer à partir de la liste des arrêts, en faisant la distinction par mode de transport.

**Il n'est pas utile de relancer ce calcul en cas de changement de date d'analyse, les tronçons restent identiques pour un même zip GTFS.**

### Création des tronçons

La cellule suivante est une copie de la routine de création des tronçons d'indicateurs. Il n'est pas nécessaire de l'exécuter, mais vous pouvez la consulter et la modifier si besoin.

In [ ]:
def creer_troncons_uniques(feed, route_type):
    """
    Crée un GeoDataFrame des tronçons uniques pour un type de route donné.
    
    Tronçons uniques = paires d'arrêts parents, tous sens confondus, sans distinction de route.
    
    Parameters:
    -----------
    feed : gtfs_kit Feed object
        Feed GTFS chargé
    route_type : int
        Type de route GTFS (0=tram, 3=bus, etc.)
    
    Returns:
    --------
    GeoDataFrame avec les tronçons uniques
    """
    print(f"\nCréation des tronçons uniques pour route_type={route_type}...")
    
    # 1. Préparer le mapping vers les parent_stations
    stops = feed.stops.copy()
    stops['parent_station'] = stops['parent_station'].fillna(stops['stop_id'])
    stops.loc[stops['parent_station'] == '', 'parent_station'] = stops['stop_id']
    
    # Mapping stop_id -> parent_station
    stop_to_parent = stops.set_index('stop_id')['parent_station'].to_dict()
    
    # Infos des parents (coords, noms)
    parent_info = stops[stops['stop_id'] == stops['parent_station']].set_index('stop_id')[
        ['stop_name', 'stop_lat', 'stop_lon']
    ].to_dict('index')
    
    # 2. Filtrer les trips du bon type de route
    routes_filtrees = feed.routes[feed.routes['route_type'] == route_type]['route_id']
    trips_filtres = feed.trips[feed.trips['route_id'].isin(routes_filtrees)]
    
    # 3. Joindre stop_times avec les trips filtrés
    stop_times = feed.stop_times.merge(
        trips_filtres[['trip_id']],
        on='trip_id',
        how='inner'
    )
    
    print(f"  → {len(trips_filtres)} trips, {len(stop_times)} stop_times")
    
    # 4. Mapper vers parent_stations
    stop_times['stop_parent'] = stop_times['stop_id'].map(stop_to_parent)
    
    # 5. Trier par trip et séquence
    stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])
    
    # 6. Créer les paires d'arrêts consécutifs
    print("  → Création des paires d'arrêts consécutifs...")
    
    # Décalage pour obtenir l'arrêt suivant
    stop_times['stop_parent_suivant'] = stop_times.groupby('trip_id')['stop_parent'].shift(-1)
    
    # Supprimer les derniers arrêts de chaque trip (pas de suivant)
    paires = stop_times.dropna(subset=['stop_parent_suivant']).copy()
    
    # 7. Créer une clé unique pour chaque paire (tous sens confondus)
    print("  → Normalisation des paires (tous sens confondus)...")
    
    paires['stop_pair'] = paires.apply(
        lambda row: tuple(sorted([row['stop_parent'], row['stop_parent_suivant']])),
        axis=1
    )
    
    # 8. Dédupliquer pour obtenir les tronçons uniques
    troncons_uniques = paires[['stop_pair']].drop_duplicates().reset_index(drop=True)
    
    print(f"  → {len(troncons_uniques)} tronçons uniques identifiés")
    
    # 9. Enrichir avec les informations des arrêts
    print("  → Enrichissement avec coordonnées et noms...")
    
    def enrichir_troncon(stop_pair):
        """Enrichit un tronçon avec les infos des deux arrêts"""
        stop1, stop2 = stop_pair
        
        # Infos du parent 1
        info1 = parent_info.get(stop1, {})
        # Infos du parent 2
        info2 = parent_info.get(stop2, {})
        
        return {
            'stop_depart_parent_id': stop1,
            'stop_arrivee_parent_id': stop2,
            'stop_depart_name': info1.get('stop_name', ''),
            'stop_arrivee_name': info2.get('stop_name', ''),
            'lat_depart_parent': info1.get('stop_lat', None),
            'lon_depart_parent': info1.get('stop_lon', None),
            'lat_arrivee_parent': info2.get('stop_lat', None),
            'lon_arrivee_parent': info2.get('stop_lon', None)
        }
    
    # Appliquer l'enrichissement
    infos_enrichies = troncons_uniques['stop_pair'].apply(enrichir_troncon)
    df_enrichi = pd.DataFrame(infos_enrichies.tolist())
    
    # Combiner avec l'index original
    troncons_uniques = pd.concat([troncons_uniques, df_enrichi], axis=1)
    
    # 10. Générer les identifiants et géométries
    print("  → Génération des identifiants et géométries...")
    
    # Identifiants uniques
    route_type_prefix = 'TRAM' if route_type == 0 else 'BUS' if route_type == 3 else f'RT{route_type}'
    troncons_uniques['troncon_unique_id'] = [
        f"TU_{route_type_prefix}_{i:06d}" for i in range(len(troncons_uniques))
    ]
    
    # Géométries LineString
    troncons_uniques['geometry'] = troncons_uniques.apply(
        lambda row: LineString([
            (row['lon_depart_parent'], row['lat_depart_parent']),
            (row['lon_arrivee_parent'], row['lat_arrivee_parent'])
        ]) if pd.notna(row['lon_depart_parent']) else None,
        axis=1
    )
    
    # 11. Créer le GeoDataFrame
    colonnes_finales = [
        'troncon_unique_id',
        'stop_depart_parent_id',
        'stop_arrivee_parent_id',
        'stop_depart_name',
        'stop_arrivee_name',
        'lat_depart_parent',
        'lon_depart_parent',
        'lat_arrivee_parent',
        'lon_arrivee_parent',
        'geometry'
    ]
    
    gdf = gpd.GeoDataFrame(
        troncons_uniques[colonnes_finales],
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Supprimer la colonne temporaire stop_pair
    if 'stop_pair' in gdf.columns:
        gdf = gdf.drop(columns=['stop_pair'])
    
    print(f"✓ {len(gdf)} tronçons uniques créés")
    
    return gdf


In [8]:
# Tronçons de bus
troncons_bus = creer_troncons_uniques(feed, route_type=3)
    
# Tronçons de bus
troncons_tram = creer_troncons_uniques(feed, route_type=0)

# Il est possible d'exporter les résultats au format csv (sans géométrie, ou en geojson)
# exporter_gdf_to_csv(troncons_bus, 'output/troncons_uniques_bus.csv')
# exporter_geojson(troncons_bus, 'output/troncons_uniques_bus.geojson')

# exporter_gdf_to_csv(troncons_tram, 'output/troncons_uniques_tram.csv')
# exporter_geojson(troncons_tram, 'output/troncons_uniques_tram.geojson')


Création des tronçons uniques pour route_type=3...
  → 9557 trips, 166826 stop_times
  → Création des paires d'arrêts consécutifs...
  → Normalisation des paires (tous sens confondus)...
  → 629 tronçons uniques identifiés
  → Enrichissement avec coordonnées et noms...
  → Génération des identifiants et géométries...
✓ 629 tronçons uniques créés

Création des tronçons uniques pour route_type=0...
  → 6951 trips, 167898 stop_times
  → Création des paires d'arrêts consécutifs...
  → Normalisation des paires (tous sens confondus)...
  → 94 tronçons uniques identifiés
  → Enrichissement avec coordonnées et noms...
  → Génération des identifiants et géométries...
✓ 94 tronçons uniques créés


### Indicateurs sur les tronçons

La cellule suivante est une copie de la routine de création des indicateurs sur les tronçons. Il n'est pas nécessaire de l'exécuter, mais vous pouvez la consulter et la modifier si besoin.

In [ ]:
def calculer_distance_haversine(lat1, lon1, lat2, lon2):
    """
    Calcule la distance entre deux points GPS en kilomètres
    Utilise la formule de Haversine
    """
    R = 6371  # Rayon de la Terre en km
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def convertir_temps_en_secondes(temps_str):
    """
    Convertit un temps GTFS (HH:MM:SS) en secondes
    Gère les heures > 24 (ex: 25:30:00 pour 01:30 le lendemain)
    """
    if pd.isna(temps_str):
        return None
    
    parts = str(temps_str).split(':')
    heures = int(parts[0])
    minutes = int(parts[1])
    secondes = int(parts[2])
    
    return heures * 3600 + minutes * 60 + secondes

def preparer_mapping_parent_stops(feed):
    """
    Crée un mapping entre stop_id et parent_station
    Si parent_station n'existe pas, utilise stop_id comme parent
    """
    stops = feed.stops.copy()
    
    # Si parent_station n'existe pas ou est vide, utiliser stop_id
    if 'parent_station' not in stops.columns:
        stops['parent_station'] = stops['stop_id']
    else:
        stops['parent_station'] = stops['parent_station'].fillna(stops['stop_id'])
        # Remplacer les chaînes vides par stop_id
        stops.loc[stops['parent_station'] == '', 'parent_station'] = stops['stop_id']
    
    return stops[['stop_id', 'parent_station']].set_index('stop_id')['parent_station'].to_dict()


def calculer_frequentation_troncons(feed, df_troncons_uniques, service_ids, route_type):
    """
    Calcule la fréquentation et la vitesse moyenne pour chaque tronçon unique
    
    Parameters:
    -----------
    feed : gtfs_kit Feed object
        Le feed GTFS chargé
    df_troncons_uniques : DataFrame
        Table des tronçons uniques avec les colonnes :
        stop_depart_parent_id, stop_arrivee_parent_id, troncon_unique_id, etc.
    service_ids : list
        Liste des service_id actifs pour la date analysée
    route_type: int
        Le type de route (0=tram, 3=bus, etc.)
    
    Returns:
    --------
    DataFrame avec fréquentation et vitesse moyenne par tronçon
    """
    print("\nCalcul de la fréquentation par tronçon unique...")

    # Créer le mapping stop_id -> parent_station
    mapping_parent = preparer_mapping_parent_stops(feed)
    
    # Filtrer les trips actifs
    trips_actifs = feed.trips[feed.trips['service_id'].isin(service_ids)].copy()

    # Restriction au bon route_type
    routes_filtrees = feed.routes[feed.routes['route_type'] == route_type]
    trips_actifs = trips_actifs[trips_actifs['route_id'].isin(routes_filtrees['route_id'])]

    print(f"✓ {len(trips_actifs)} trips actifs")
    
    # Enrichir stop_times avec les informations nécessaires
    stop_times = feed.stop_times.merge(
        trips_actifs[['trip_id', 'route_id']], 
        on='trip_id'
    )
    
    # Ajouter les parent_station pour chaque stop
    stop_times['stop_parent_id'] = stop_times['stop_id'].map(mapping_parent)
    
    # Trier par trip et séquence
    stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])
    
    print(f"✓ {len(stop_times)} stop_times à analyser")
    
    # Calculer les passages par paire de stops consécutifs
    passages_list = []
    
    for trip_id, groupe in stop_times.groupby('trip_id'):
        groupe = groupe.sort_values('stop_sequence').reset_index(drop=True)
        
        for i in range(len(groupe) - 1):
            depart = groupe.iloc[i]
            arrivee = groupe.iloc[i + 1]
            
            # Calculer le temps de parcours
            temps_depart = convertir_temps_en_secondes(depart['departure_time'])
            temps_arrivee = convertir_temps_en_secondes(arrivee['arrival_time'])
            
            if temps_depart is not None and temps_arrivee is not None:
                duree_secondes = temps_arrivee - temps_depart
                
                if duree_secondes > 0:
                    # Identifier le tronçon (tous sens confondus)
                    parent_depart = depart['stop_parent_id']
                    parent_arrivee = arrivee['stop_parent_id']
                    
                    # Créer une clé normalisée (ordre alphabétique pour regrouper les deux sens)
                    stops_pair = tuple(sorted([parent_depart, parent_arrivee]))
                    
                    passages_list.append({
                        'stop_pair': stops_pair,
                        'stop_depart_parent': parent_depart,
                        'stop_arrivee_parent': parent_arrivee,
                        'duree_secondes': duree_secondes,
                        'trip_id': trip_id
                    })
    
    print(f"✓ {len(passages_list)} passages détectés")
    
    if not passages_list:
        print("⚠ Aucun passage détecté")
        return None
    
    df_passages = pd.DataFrame(passages_list)
    
    # Agréger par paire de stops (tous sens confondus)
    # On compte le nombre de passages et calcule la durée moyenne
    stats_par_paire = df_passages.groupby('stop_pair').agg(
        nombre_passages=('trip_id', 'count'),
        duree_moyenne_secondes=('duree_secondes', 'mean'),
        duree_min_secondes=('duree_secondes', 'min'),
        duree_max_secondes=('duree_secondes', 'max')
    ).reset_index()
    
    print(f"✓ Statistiques calculées pour {len(stats_par_paire)} paires de stops")
    
    # Préparer le matching avec df_troncons_uniques
    # Créer la même clé normalisée dans df_troncons_uniques
    df_troncons_uniques['stop_pair'] = df_troncons_uniques.apply(
        lambda row: tuple(sorted([row['stop_depart_parent_id'], row['stop_arrivee_parent_id']])),
        axis=1
    )
    
    # Joindre avec les statistiques
    df_resultat = df_troncons_uniques.merge(
        stats_par_paire,
        on='stop_pair',
        how='left'
    )
    
    # Supprimer la colonne temporaire
    df_resultat = df_resultat.drop(columns=['stop_pair'])
    
    # Calculer la distance si pas déjà présente
    if 'distance_km' not in df_resultat.columns:
        df_resultat['distance_km'] = df_resultat.apply(
            lambda row: calculer_distance_haversine(
                row['lat_depart_parent'], row['lon_depart_parent'],
                row['lat_arrivee_parent'], row['lon_arrivee_parent']
            ),
            axis=1
        )
    
    # Calculer la vitesse moyenne en km/h
    df_resultat['vitesse_moyenne_kmh'] = (
        df_resultat['distance_km'] / 
        (df_resultat['duree_moyenne_secondes'] / 3600)
    )
    
    # Calculer aussi vitesse min et max
    df_resultat['vitesse_min_kmh'] = (
        df_resultat['distance_km'] / 
        (df_resultat['duree_max_secondes'] / 3600)
    )
    
    df_resultat['vitesse_max_kmh'] = (
        df_resultat['distance_km'] / 
        (df_resultat['duree_min_secondes'] / 3600)
    )
    
    # Remplacer les NaN (tronçons sans passage) par 0
    df_resultat['nombre_passages'] = df_resultat['nombre_passages'].fillna(0).astype(int)
    
    # Trier par nombre de passages décroissant
    df_resultat = df_resultat.sort_values('nombre_passages', ascending=False).reset_index(drop=True)
    
    print(f"✓ Fréquentation calculée pour {len(df_resultat)} tronçons uniques")
    print(f"✓ Tronçons avec passages : {(df_resultat['nombre_passages'] > 0).sum()}")
    
    return df_resultat

def compute_indicateurs_troncons(
        feed,
        active_service_ids: list[str],
        reference_troncons_uniques_bus: pd.DataFrame,
        reference_troncons_uniques_tram: pd.DataFrame
    ):
    """
    Réalise le calcul des indicateurs par tronçon pour une date d'analyse donnée
    Args:
        feed: gtfs_kit Feed object
            Le feed GTFS chargé
        active_service_ids: list[str]: Liste des services actifs à la date choisie
        reference_troncons_uniques_bus (pd.DataFrame):
            Table des tronçons uniques bus
        reference_troncons_uniques_tram (pd.DataFrame):
            Table des tronçons uniques tram
    
    Returns:
        Tuple de GeoDataFrame : (indicateurs_bus, indicateurs_tram)
    """
    
    # Calculer la fréquentation
    indicateurs_bus = calculer_frequentation_troncons(
        feed, 
        reference_troncons_uniques_bus, 
        active_service_ids,
        route_type=3  # Bus
    )

    indicateurs_tram = calculer_frequentation_troncons(
        feed, 
        reference_troncons_uniques_tram, 
        active_service_ids,
        route_type=0  # Tram
    )

    # Convertir en GeoDataFrame
    indicateurs_bus_gdf = gpd.GeoDataFrame(indicateurs_bus, geometry='geometry', crs='EPSG:4326')

    indicateurs_tram_gdf = gpd.GeoDataFrame(indicateurs_tram, geometry='geometry', crs='EPSG:4326')
    
    return indicateurs_bus_gdf, indicateurs_tram_gdf


In [9]:
# Les indicateurs sont calculés sous la forme d'un GeoDataframe par mode de transport

indicateurs_bus, indicateurs_tram = compute_indicateurs_troncons(
    feed,  # le feed GTFS chargé plus haut
    active_service_ids,  # prise en compte uniquement des services actifs ce jour
    troncons_bus,  # Geodataframe des tronçons de bus
    troncons_tram  # Geodataframe des tronçons de tram
)

# Il est possible d'exporter les résultats au format csv (sans géométrie, ou en geojson)
# exporter_gdf_to_csv(indicateurs_bus, f'output/indicateurs_troncons_bus_{DATE_ANALYSE}.csv')
# exporter_gdf_to_csv(indicateurs_tram, f'output/indicateurs_troncons_tram_{DATE_ANALYSE}.csv')

# exporter_geojson(indicateurs_bus, f'output/indicateurs_troncons_bus_{DATE_ANALYSE}.geojson')
# exporter_geojson(indicateurs_tram, f'output/indicateurs_troncons_tram_{DATE_ANALYSE}.geojson')


Calcul de la fréquentation par tronçon unique...
✓ 935 trips actifs
✓ 18026 stop_times à analyser
✓ 13911 passages détectés
✓ Statistiques calculées pour 575 paires de stops
✓ Fréquentation calculée pour 629 tronçons uniques
✓ Tronçons avec passages : 575

Calcul de la fréquentation par tronçon unique...
✓ 561 trips actifs
✓ 13855 stop_times à analyser
✓ 13245 passages détectés
✓ Statistiques calculées pour 94 paires de stops
✓ Fréquentation calculée pour 94 tronçons uniques
✓ Tronçons avec passages : 94


### Représentation cartographique

Les tronçons sont représentés sur une carte et colorés en fonction de leur fréquentation

In [ ]:
def creer_carte_troncons(gdf_bus, gdf_tram, colonne_frequence='nombre_passages'):
    """
    Crée une carte Folium interactive avec les tronçons bus et tram.
    Les tronçons sont colorés selon la fréquence et peuvent être activés/désactivés.
    
    Parameters:
    -----------
    gdf_bus : GeoDataFrame
        GeoDataFrame des tronçons bus avec indicateurs
    gdf_tram : GeoDataFrame
        GeoDataFrame des tronçons tram avec indicateurs
    colonne_frequence : str
        Nom de la colonne contenant la fréquence (défaut: 'nombre_passages')
    
    Returns:
    --------
    folium.Map
        Carte Folium interactive
    """
    
    # Déterminer le centre de la carte (moyenne des coordonnées)
    all_coords = []
    for gdf in [gdf_bus, gdf_tram]:
        if len(gdf) > 0:
            all_coords.extend(gdf['lat_depart_parent'].dropna().tolist())
            all_coords.extend(gdf['lat_arrivee_parent'].dropna().tolist())
    
    if not all_coords:
        center_lat, center_lon = 45.75, 4.85  # Lyon par défaut
    else:
        center_lat = np.mean(all_coords)
        all_lons = []
        for gdf in [gdf_bus, gdf_tram]:
            if len(gdf) > 0:
                all_lons.extend(gdf['lon_depart_parent'].dropna().tolist())
                all_lons.extend(gdf['lon_arrivee_parent'].dropna().tolist())
        center_lon = np.mean(all_lons)
    
    # Créer la carte de base
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    # Ajouter des fonds de carte alternatifs
    folium.TileLayer('cartodbpositron', name='Carto Positron').add_to(m)
    folium.TileLayer('cartodbdark_matter', name='Carto Dark').add_to(m)
    
    # ===== TRONÇONS BUS =====
    if len(gdf_bus) > 0 and colonne_frequence in gdf_bus.columns:
        # Filtrer les tronçons avec passages
        gdf_bus_actif = gdf_bus[gdf_bus[colonne_frequence] > 0].copy()
        
        if len(gdf_bus_actif) > 0:
            # Créer la palette de couleurs pour les bus
            vmin_bus = gdf_bus_actif[colonne_frequence].min()
            vmax_bus = gdf_bus_actif[colonne_frequence].max()
            
            colormap_bus = cm.LinearColormap(
                colors=['#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15'],
                vmin=vmin_bus,
                vmax=vmax_bus,
                caption=f'fréquence Bus (passages)'
            )
            
            # Créer un groupe de features pour les bus
            feature_group_bus = folium.FeatureGroup(name='🚌 Bus', show=True)
            
            # Ajouter chaque tronçon bus
            for idx, row in gdf_bus_actif.iterrows():
                freq = row[colonne_frequence]
                color = colormap_bus(freq)
                
                # Extraire les coordonnées de la géométrie
                coords = [(coord[1], coord[0]) for coord in row['geometry'].coords]
                
                # Créer le popup avec les informations
                popup_html = f"""
                <div style="font-family: Arial; font-size: 12px; width: 250px;">
                    <b style="color: #d63447;">🚌 TRONÇON BUS</b><br>
                    <hr style="margin: 5px 0;">
                    <b>ID:</b> {row.get('troncon_unique_id', 'N/A')}<br>
                    <b>De:</b> {row.get('stop_depart_name', 'N/A')}<br>
                    <b>À:</b> {row.get('stop_arrivee_name', 'N/A')}<br>
                    <hr style="margin: 5px 0;">
                    <b>Passages:</b> {int(freq)}<br>
                    <b>Vitesse moy.:</b> {row.get('vitesse_moyenne_kmh', 0):.1f} km/h<br>
                    <b>Distance:</b> {row.get('distance_km', 0):.2f} km
                </div>
                """
                
                # Épaisseur proportionnelle à la fréquence
                weight = 2 + (freq - vmin_bus) / (vmax_bus - vmin_bus) * 6
                
                folium.PolyLine(
                    coords,
                    color=color,
                    weight=weight,
                    opacity=0.8,
                    popup=folium.Popup(popup_html, max_width=300),
                    tooltip=f"{row.get('stop_depart_name', '')} → {row.get('stop_arrivee_name', '')}: {int(freq)} passages"
                ).add_to(feature_group_bus)
            
            feature_group_bus.add_to(m)
            colormap_bus.add_to(m)
    
    # ===== TRONÇONS TRAM =====
    if len(gdf_tram) > 0 and colonne_frequence in gdf_tram.columns:
        # Filtrer les tronçons avec passages
        gdf_tram_actif = gdf_tram[gdf_tram[colonne_frequence] > 0].copy()
        
        if len(gdf_tram_actif) > 0:
            # Créer la palette de couleurs pour les trams
            vmin_tram = gdf_tram_actif[colonne_frequence].min()
            vmax_tram = gdf_tram_actif[colonne_frequence].max()
            
            colormap_tram = cm.LinearColormap(
                colors=['#edf8e9', '#bae4b3', '#74c476', '#31a354', '#006d2c'],
                vmin=vmin_tram,
                vmax=vmax_tram,
                caption=f'fréquence Tram (passages)'
            )
            
            # Créer un groupe de features pour les trams
            feature_group_tram = folium.FeatureGroup(name='🚊 Tram', show=True)
            
            # Ajouter chaque tronçon tram
            for idx, row in gdf_tram_actif.iterrows():
                freq = row[colonne_frequence]
                color = colormap_tram(freq)
                
                # Extraire les coordonnées de la géométrie
                coords = [(coord[1], coord[0]) for coord in row['geometry'].coords]
                
                # Créer le popup avec les informations
                popup_html = f"""
                <div style="font-family: Arial; font-size: 12px; width: 250px;">
                    <b style="color: #28a745;">🚊 TRONÇON TRAM</b><br>
                    <hr style="margin: 5px 0;">
                    <b>ID:</b> {row.get('troncon_unique_id', 'N/A')}<br>
                    <b>De:</b> {row.get('stop_depart_name', 'N/A')}<br>
                    <b>À:</b> {row.get('stop_arrivee_name', 'N/A')}<br>
                    <hr style="margin: 5px 0;">
                    <b>Passages:</b> {int(freq)}<br>
                    <b>Vitesse moy.:</b> {row.get('vitesse_moyenne_kmh', 0):.1f} km/h<br>
                    <b>Distance:</b> {row.get('distance_km', 0):.2f} km
                </div>
                """
                
                # Épaisseur proportionnelle à la fréquence
                weight = 2 + (freq - vmin_tram) / (vmax_tram - vmin_tram) * 6
                
                folium.PolyLine(
                    coords,
                    color=color,
                    weight=weight,
                    opacity=0.8,
                    popup=folium.Popup(popup_html, max_width=300),
                    tooltip=f"{row.get('stop_depart_name', '')} → {row.get('stop_arrivee_name', '')}: {int(freq)} passages"
                ).add_to(feature_group_tram)
            
            feature_group_tram.add_to(m)
            colormap_tram.add_to(m)
    
    # Ajouter le contrôle des couches (cases à cocher)
    folium.LayerControl(collapsed=False).add_to(m)
    
    # Ajouter un bouton plein écran
    plugins.Fullscreen(
        position='topright',
        title='Plein écran',
        title_cancel='Quitter le plein écran',
        force_separate_button=True
    ).add_to(m)
    
    # Ajouter la mesure de distance
    plugins.MeasureControl(
        position='topleft',
        primary_length_unit='kilometers',
        secondary_length_unit='meters'
    ).add_to(m)
    
    return m


In [10]:
# Créer la carte des tronçons
carte_troncons = creer_carte_troncons(indicateurs_bus, indicateurs_tram, colonne_frequence='nombre_passages')

# Afficher la carte
carte_troncons

In [ ]:
# (Optionnel) Sauvegarder en HTML
carte_troncons.save(f'output/ma_carte_troncons_{DATE_ANALYSE}.html')